In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
WORK_DIR = '/content/drive/My Drive/datasets'
DATA_DIR = os.path.join(WORK_DIR, 'olist')

In [ ]:
df_abt = pd.read_csv(os.path.join(DATA_DIR, 'propensao_revenda_abt.csv'))

In [ ]:
df_abt.head()

,data_ref_safra,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia,nao_revendeu_next_6m
0,2018-01-01,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,74,1
1,2018-01-01,001cca7ae9ae17fb1caed9dfb1094831,ES,171,207,9,21275.23,2,0
2,2018-01-01,002100f778ceb8431b7a1020ff7ab48f,SP,38,42,15,781.80,2,0
3,2018-01-01,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,16,1
4,2018-01-01,004c9cd9d87a3c30c522c48c4fc07416,SP,130,141,75,16228.88,8,0


## Identificar as variáveis de modelagem

In [ ]:
# Variáveis chaves da tabela
key_vars = ['data_ref_safra', 'seller_id']

# Variáveis numéricas
num_vars = ['tot_orders_12m', 'tot_items_12m', 'tot_items_dist_12m', 'receita_12m', 'recencia']

# Variáveis categóricas
cat_vars = ['uf']

# Variável resposta/target
target = 'nao_revendeu_next_6m'

# criando a lista com as features
features = cat_vars + num_vars

# Filtrando a base de features
X = df_abt[features]

# Filtrando o target
y = df_abt[target]

In [ ]:
X.head()

,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia
0,SP,3,3,1,2685.00,74
1,ES,171,207,9,21275.23,2
2,SP,38,42,15,781.80,2
3,GO,1,1,1,120.00,16
4,SP,130,141,75,16228.88,8


In [ ]:
y.head()

0    1
1    0
2    0
3    1
4    0
Name: nao_revendeu_next_6m, dtype: int64

# Treinando primeiros modelos

## Árvore de Decisão

Como sabemos as variáveis categóricas precisam ser transformadas, porém para esse primeiro teste vamos  utilizar apenas as variáveis numéricas.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_model = DecisionTreeClassifier(random_state=1)

In [ ]:
dt_model.fit(X[num_vars], y)

DecisionTreeClassifier(random_state=1)

In [ ]:
dt_model.predict(X[num_vars])

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
from sklearn.metrics import accuracy_score

# Avaliando o modelo na base
acc_train_dt = accuracy_score(y, dt_model.predict(X[num_vars]))
acc_train_dt

0.9998137455764574

**ATENÇÃO**

Nesse primeiro modelo, mesmo tendo uma acurácia boa, é importante salientar que pulamos algumas etapas:

- Imputação de Missing
- Feature Scaling -> só para modelos lineares, exeção para modelos baseados em árvores.
- Tratamento das variáveis categóricas
- Separar a base em treino e teste.


## Meu algoritmo supimpa

In [ ]:
import random

class MeuModeloSupimpa:
    def __init__(self, random_state=42):
        self.random_state = random_state

    def fit(self, X, y):
        return None

    def predict(self, X):
        random.seed(self.random_state)
        return random.choices([0, 1], k=len(X))

In [ ]:
meu_modelo = MeuModeloSupimpa()
meu_modelo.fit(X, y)

In [ ]:
acc_train_mms = accuracy_score(y, meu_modelo.predict(X[num_vars]))
acc_train_mms

0.49301545911715405

# Bias-Variance Tradeoff

Para entendermos melhor esses conceitos, iremos dividir aleatoriamente o conjunto de dados para gerar 2 grupos: um para **treinar** (80%) o modelo e outro para **avaliar** (20%) o desempenho.

Utilizamos a função `train_test_split()` do módulo `model_selection` da biblioteca `sklearn`.

* Parâmetros

 - `train_size`: percentual da base que irá ser utilizado para treinamento do modelo.
 - `stratify`: faz  uma amostragem aleatória estratificada, mantendo a mesma distribuição da variável target para ambos os datasets de treino e teste.
 - `random_state`: Controla o embaralhamento dos dados antes de se fazer o split. Utilizando o mesmo valor inteiro para esse parâmetro em várias chamadas da função `train_test_split`, garantimos a reproducibilidade da amostragem.




In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_abt.drop('nao_revendeu_next_6m', axis=1).copy()
y = df_abt['nao_revendeu_next_6m'].copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                                X[num_vars],
                                                y,
                                                train_size=0.8,
                                                stratify=y,
                                                random_state=55
                                    )

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4295, 5), (1074, 5), (4295,), (1074,))

Vamos retreinar e avaliar os modelos que testamos no tópico anterior.

## Underfitting

O modelo gerado não se ajusta aos dados e portanto o modelo não tem a expressividade suficiente para ajustar os dados.

In [ ]:
mms_model = MeuModeloSupimpa(random_state=42)

mms_model.fit(X_train, y_train)

acc_train = accuracy_score(y_train, mms_model.predict(X_train))
acc_test = accuracy_score(y_test, mms_model.predict(X_test))

acc_train = accuracy_score(y_train, mms_model.predict(X_train))
acc_test = accuracy_score(y_test, mms_model.predict(X_test))

Acurácia Treino: 50.20%
Acurácia Teste: 50.74%


Podemos dizer que esse modelo tem alto viés (bias) e baixa variância (variance).

- Tem alto viés pois o modelo simples não tem a capacidade de capturar a relação verdadeira entre os dados e o algoritmo.

- Tem baixa variância pois existe pouco diferença entre as predições no treino e teste.

## Overfitting

O modelo gerado tem muita expressividade e "memoriza" os dados de treinamento. O que buscamos em Machine Learning é sempre ter uma boa generalização.

Portanto, dizemos que esse modelo sobre ajusta (overfitting) aos dados.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_model = DecisionTreeClassifier(random_state=42)

dt_model.fit(X_train, y_train)

acc_train = accuracy_score(y_train, dt_model.predict(X_train))
acc_test = accuracy_score(y_test, dt_model.predict(X_test))

print(f'Acurácia Treino: {100 * acc_train:.2f}%')
print(f'Acurácia Teste: {100 * acc_test:.2f}%')

Acurácia Treino: 99.98%
Acurácia Teste: 77.56%


Podemos dizer que esse modelo tem baixo viés (bias) e alta variância (variance).

- Esse modelo conseguiu lidar melhor com a relação verdadeira entre os dados e o algoritmo e portante tem baixo viés.
- Tem alta variância pois existe grande diferença entre as predições no treino e teste.